In [3]:
!pip install calplot
!pip install folium

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import geopandas as gpd

import statsmodels.api as sm
import calplot
import folium
from folium.plugins import HeatMapWithTime

In [6]:
df = pd.read_csv('us_congestion_2016_2022.csv')

C:\Users\U437186\AppData\Local\Temp\ipykernel_24700\1785735340.py:1: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('us_congestion_2016_2022.csv')


In [27]:
cities = ["New York", "Los Angeles"]

In [43]:
df_filt = df[df["City"].isin(cities)]

In [29]:
df_filt.to_csv(r"C:\Users\U437186\OneDrive - Danfoss\Desktop\Mona\DTU\Social data\Assignment 2\Project B\Data filtered city.csv", index = False)

# Preprocess

How many NaN values do we have?

In [57]:
# 1. Total number of rows with at least one NaN
num_rows_with_nan = df_filt.isna().any(axis=1).sum()
print(f"Rows with at least one NaN: {num_rows_with_nan}")

# 2. Number of NaNs in each column
nan_per_column = df_filt.isna().sum()
print("\nNaNs per column:")
print(nan_per_column)

Rows with at least one NaN: 1578479

NaNs per column:
ID                                     0
Severity                               0
Start_Lat                              0
Start_Lng                              0
StartTime                              0
EndTime                                0
Distance(mi)                           0
DelayFromTypicalTraffic(mins)          0
DelayFromFreeFlowSpeed(mins)           0
Congestion_Speed                       0
Description                            0
Street                                 0
City                                   0
County                                 0
State                                  0
Country                                0
ZipCode                                0
LocalTimeZone                          0
WeatherStation_AirportCode             0
WeatherTimeStamp                    9061
Temperature(F)                     14940
WindChill(F)                      639579
Humidity(%)                        15146
Pre

We choose to only use the weather attributes, WeatherTimeStamp, Temperature, Humidity, Pressure, Visibility, WindDir, **WindSpeed** (Reconsider this), Weather_Conditions

**Convert to proper datetime**

Remove the time offset

In [ ]:
df_filt["StartTime"] = df_filt["StartTime"].str[:-10] #Remove the time offset from UTC

df_filt["StartTime"] = pd.to_datetime(df_filt["StartTime"], format="%Y-%m-%dT%H:%M:%S") # Convert to datetime finally

C:\Users\U437186\AppData\Local\Temp\ipykernel_24700\2115086838.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filt["StartTime"] = pd.to_datetime(df_filt["StartTime"], format="%Y-%m-%dT%H:%M:%S")


In [92]:
columns_to_use1 = list(df_filt.columns[:21]) + ["Humidity(%)", "Pressure(in)", "Visibility(mi)",  "WindDir", "WindSpeed(mph)", "Weather_Conditions"] # With windspeed
columns_to_use2 = list(df_filt.columns[:21]) + ["Humidity(%)", "Pressure(in)", "Visibility(mi)",  "WindDir", "Weather_Conditions"] # Without windspeed

In [94]:
df2 = df_filt[columns_to_use1]
df_noWindspeed = df_filt[columns_to_use2]


df3 = df2.dropna()
df_noWindspeed_filt = df_noWindspeed.dropna()

In [97]:
print("Length before dropping windspeed", len(df2))
print("Length after dropping with windspeed", len(df3))
print("Length of dataframe without windspeed after dropping", len(df_noWindspeed_filt))

Length before dropping windspeed 1584650
Length after dropping with windspeed 1289973
Length of dataframe without windspeed after dropping 1509892


In [98]:
df_NY_prev = df2[df2["City"] == "New York"]
df_LA_prev = df2[df2["City"] == "Los Angeles"]

df_NY_after = df3[df3["City"] == "New York"]
df_LA_after = df3[df3["City"] == "Los Angeles"]

print("Length of NY before dropping", len(df_NY_prev))
print("Length of LA before dorpping", len(df_LA_prev))

print("Length of NY after dropping", len(df_NY_after))
print("Length of LA after dorpping", len(df_LA_after))


Length of NY before dropping 660245
Length of LA before dorpping 924405
Length of NY after dropping 523531
Length of LA after dorpping 766442


In [99]:
# Save the dataframe that contains windspeed (ends up dropping around 300k observations)
df3.to_csv(r"C:\Users\U437186\OneDrive - Danfoss\Desktop\Mona\DTU\Social data\Assignment 2\Project B\Car Congestion with windspeed.csv", index = False)
df_noWindspeed_filt.to_csv(r"C:\Users\U437186\OneDrive - Danfoss\Desktop\Mona\DTU\Social data\Assignment 2\Project B\Car Congestion without windspeed.csv", index = False)

## **<div style="text-align: center;">Part 1 - Temporal patterns</div>**

* **To start off easily, let's count the number of congestions per year:**
    * **What is the year with most congestions?**
    * **What is the year with the fewest congestions?.**

In [ ]:
# Group data by 'year' and 'state' and count occurrences
grouped = df_filt.groupby(['hour', 'Category']).size().unstack(fill_value=0)